In [1]:
import json
import csv
import os
import glob
import pandas as pd

In [2]:
with open("../visData.json") as f:
    data = json.load(f)

In [3]:
file_names =  []

for item in data:
    file_names.append('..' + item['Highlight Data'])

In [4]:
combined_csv = pd.concat([pd.read_csv(f) for f in file_names ])

combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [6]:
combined = pd.read_csv('combined_csv.csv')

In [17]:
combined.head(3)

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN


In [18]:
combined[['Start','End']]

,Start,End
0,-1.0,-1.0
1,-1.0,-1.0
2,-1.0,-1.0
3,-1.0,-1.0
4,-1.0,-1.0
...,...,...
118,-1.0,-1.0
119,-1.0,-1.0
120,-1.0,-1.0
121,3017.0,3197.0


In [19]:
combined['Start,End'] = combined[['Start','End']].apply(tuple,axis = 1)

In [20]:
combined.head(3)

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start,End"
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)"
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)"
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)"


In [21]:
combined.groupby(['Article ID', 'Credibility Indicator Category','Credibility Indicator Name','Start,End']).count()

Credibility Indicator ID  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Start,End                                    
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           (-1.0, -1.0)                             1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)                             4   
                                                                                  Vaguely Sourced            (-1.0, -1.0)                             1   
                                                   Language                       Metaphor unhelpful         (387.0, 398.0)                           1   
                                                   Probability                    Acknowledges uncertainty   (259.0, 268.0)                           1   
...                                                                                                                                                 ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Evidence                       Open to evidence           (1805.0, 1815.0)                         1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)                             4   
                                                                                  Vague Sourcing             (-1.0, -1.0)                             1   
                                                   Probability                    Acknowledges uncertainty   (2204.0, 2389.0)                         1   
                                                                                  Open to evidence           (2274.0, 2496.0)                         1   

                                                                                                                               Points  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Start,End                  
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           (-1.0, -1.0)           1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)           4   
                                                                                  Vaguely Sourced            (-1.0, -1.0)           1   
                                                   Language                       Metaphor unhelpful         (387.0, 398.0)         1   
                                                   Probability                    Acknowledges uncertainty   (259.0, 268.0)         1   
...                                                                                                                               ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Evidence                       Open to evidence           (1805.0, 1815.0)       1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)           4   
                                                                                  Vague Sourcing             (-1.0, -1.0)           1   
                                                   Probability                    Acknowledges uncertainty   (2204.0, 2389.0)       1   
                                                                                  Open to evidence           (2274.0, 2496.0)       1   

                                                                                                                               Indices of Label in Article  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Start,End                                       
3be14d67e2d88

In [15]:
!ls

09a894f06d1157dd7a22a198c6ef78c2 assets
3be14d67e2d88964904dcbe7df176bb8 be0b18a87d4370fa579180ef26dcb708
47990959103662e94e796d979018922a combined_csv.csv
7360da3cdcf83a48e365821654ef0750 test.ipynb
7d1747829a65cd4dad0faf30a704875a test.py
